In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
from specim_test.specim.specfuncs import spec2d
from astropy.io import fits as pf

In [2]:
file_loc = '../galaxy_spectra/LRIS_red_data_20201120/keck_lris_red_B/Science/*'

In [3]:
file_list = []
out_file = []
for i,p in enumerate(glob.glob(file_loc)):
    if 'spec2d' in p and 'J0248+1913' in p:
        file_list.append(p)
        frame = p.split('_')[-3][:4]
        out_file.append('J0248_r'+ frame + '_skysub.fits')

In [4]:
for i, p in enumerate(file_list):
    spec = spec2d.Spec2d(p, hext=12, xtrim=[30, 220])
    spec.set_dispaxis('y')
    spec.subtract_sky_2d(use_skymod=True, outfile=out_file[i])


Loading file ../galaxy_spectra/LRIS_red_data_20201120/keck_lris_red_B/Science/spec2d_r201121_0037-J0248+1913_LRISr_2020Nov21T095645.974.fits
-----------------------------------------------
Read in 2-dimensional spectrum from ../galaxy_spectra/LRIS_red_data_20201120/keck_lris_red_B/Science/spec2d_r201121_0037-J0248+1913_LRISr_2020Nov21T095645.974.fits (HDU=12)
The input dataset was trimmed
 xrange: 30:221.  yrange: 0:4096
Final data dimensions (x y): 191 x 4096

Dispersion axis:              x
N_pixels along dispersion axis: 191


Old value of dispaxis: x

Dispersion axis:              y
N_pixels along dispersion axis: 4096



pypeit generated sky model will be used for sky subtraction

 Wrote sky subtracted data to J0248_r0037_skysub.fits

From this point sky subtracted data will be used
 Spectrum Start:     0.00
 Spectrum End:     4095.00
 Dispersion (1st pixel):   1.00
 Dispersion (average):      1.00


Loading file ../galaxy_spectra/LRIS_red_data_20201120/keck_lris_red_B/Science/sp

In [ ]:
for i, p in enumerate(file_list):
    spec = spec2d.Spec2d(p, hext=12, xtrim=[30, 220])
    spec.set_dispaxis('y')
    spec.subtract_sky_2d(use_skymod=True)      #, outfile=out_file[i])
    #spec.szap(use_skymod=True, outfile=out_file[i])
    filled_data, mask = szap(spec.data)
    #pf.PrimaryHDU(filled_data).writeto(out_file[i])
    
    wav_im = pf.open(p)[19].data[spec.ymin:spec.ymax, spec.xmin:spec.xmax]
    sky_im = pf.open(p)[14].data[spec.ymin:spec.ymax, spec.xmin:spec.xmax]
    ivar_im = pf.open(p)[13].data[spec.ymin:spec.ymax, spec.xmin:spec.xmax]
    #print(ivar_im.shape)
    prim_hdr = pf.Header()
    prim_hdr.append(('OBJECT', 'J0248+1913'))
    prim_hdr.append(('', 'Trimmed and cleaned data by Pritom'))
    phdu = pf.PrimaryHDU(header=prim_hdr)
    
    sci_hdr = pf.Header()
    sci_hdr.append(('IMTYPE', 'SCIENCE_IMAGE'))
    sci_im_hdu = pf.ImageHDU(data=filled_data, header=sci_hdr, name='SCI_IM')
    
    ivar_hdr = pf.Header()
    ivar_hdr.append(('IMTYPE', 'IVAR_IMAGE'))
    ivar_im_hdu = pf.ImageHDU(data=ivar_im, header=ivar_hdr, name='IVAR')
    
    sky_hdr = pf.Header()
    sky_hdr.append(('IMTYPE', 'SKY_MODEL'))
    sky_im_hdu = pf.ImageHDU(data=sky_im, header=sky_hdr, name='SKY_MOD')
    
    wav_hdr = pf.Header()
    wav_hdr.append(('IMTYPE', 'WAV_IMAGE'))
    wav_im_hdu = pf.ImageHDU(data=wav_im, header=wav_hdr, name='WAV_IM')
    
    mask_hdr = pf.Header()
    mask_hdr.append(('IMTYPE', 'MASK'))
    cmt = 'pixels masked as cosmic rays and filled with pixel values at same pixel'\
        'location from median image'
    mask_hdr.append(('', cmt))
    mask_hdu = pf.ImageHDU(data=mask, header=mask_hdr, name='MASK')
    
    hdul = pf.HDUList([phdu, sci_im_hdu, ivar_im_hdu, sky_im_hdu, wav_im_hdu, mask_hdu ])
    hdul.writeto(out_file[i])

In [ ]:
def szap(sp_1):

    sp_2 = pf.open('J0248_median_image.fits')[0].data
    mask = np.zeros(sp_1.shape, dtype=bool)
    #print(sp_1.shape)

    for k in range(0, sp_1.shape[0], 400):
        for i in range(0, sp_1.shape[1], 5):
            sp1 = sp_1[k:k+400, i:i+5]
            sp2 = sp_2[k:k+400, i:i+5]
            count1, bins1 = np.histogram(sp1.flatten(), bins=1000)
            count2, bins2 = np.histogram(sp2.flatten(), bins=1000)
            sm1 = 0
            sm2 = 0
            cnt1 = 0
            cnt2 = 0
            tot_pix =sp1.flatten().shape[0]

            for j,p in enumerate(count1):
                if p > cnt1:
                    cnt1 = p
                    max_bin1 = bins1[j+1]
                sm1 += p
                if sm1/tot_pix >= 0.8 :
                    border1 = 0.5*(bins1[j+1] + bins1[j])
                    break
            #print(border)
            for j,p in enumerate(count2):
                if p > cnt2:
                    cnt2 = p
                    max_bin2 = 0.5*(bins2[j+1] + bins2[j])
                sm2 += p
                if sm2/tot_pix >= 0.8 :
                    border2 = bins2[j+1]
                    break

            a = np.where(sp1>border1)
            a =np.transpose(a)

            if abs(border1 - max_bin1) < 50 :
                mp = 50    #/ abs(border1 - max_bin1)
            elif abs(border1 - max_bin1) < 100 :
                mp = 150
            elif abs(border1 - max_bin1) < 300:
                mp = 200
            else:
                mp = 400

            for j, p in enumerate(a):

                #print(sp2[p[0]][p[1]])
                if sp2[p[0]][p[1]] >0 :
                    #pass 
                    if sp1[p[0]][p[1]] > (sp2[p[0]][p[1]]+mp) :
                        mask[k+p[0]][i+p[1]] =1 
                else:
                    if sp1[p[0]][p[1]] > 3*border1 :
                        mask[k+p[0]][i+p[1]] =1
                        
    #print(mask)
    sp_1[mask] = sp_2[mask]
    
    mf =  (sp_1 > 500) | (sp_1 < -200)
    
    for k, p in enumerate(np.transpose(np.where(mf==True))):
        mask[p[0]][p[1]] = 1
        reg = sp_1[p[0]-10: p[0]+10, p[1]:p[1]+1]
        sp_1[p[0]][p[1]] = np.median(reg)
                
        #d[mask] = fill[mask] 
        #mm = m.astype('bool') | mask
    
    return sp_1, mask.astype(int)

In [ ]:
## median image
spec_list = []
for i, p in enumerate(file_list):
    spec = spec2d.Spec2d(p, hext=12, xtrim=[30, 220])
    spec.set_dispaxis('y')
    spec.subtract_sky_2d(use_skymod=True)
    spec_list.append(spec.data)
median_im = np.median(np.asarray(spec_list), axis=0)

In [ ]:
pf.PrimaryHDU(median_im).writeto('J0248_median_image.fits')

In [2]:
inf = '../spec2d_r201121_0037-J0248+1913_LRISr_2020Nov21T095645.974.fits'

In [3]:
spec = spec2d.Spec2d(inf, hext=12, xtrim=[30, 220])
spec.set_dispaxis('y')
spec.subtract_sky_2d(use_skymod=True, outfile='skysub_37-J0248.fits')


Loading file ../spec2d_r201121_0037-J0248+1913_LRISr_2020Nov21T095645.974.fits
-----------------------------------------------
Read in 2-dimensional spectrum from ../spec2d_r201121_0037-J0248+1913_LRISr_2020Nov21T095645.974.fits (HDU=12)
The input dataset was trimmed
 xrange: 30:221.  yrange: 0:4096
Final data dimensions (x y): 191 x 4096

Dispersion axis:              x
N_pixels along dispersion axis: 191


Old value of dispaxis: x

Dispersion axis:              y
N_pixels along dispersion axis: 4096



pypeit generated sky model will be used for sky subtraction

 Wrote sky subtracted data to skysub_37-J0248.fits

From this point sky subtracted data will be used
 Spectrum Start:     0.00
 Spectrum End:     4095.00
 Dispersion (1st pixel):   1.00
 Dispersion (average):      1.00

